1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# сигмоида
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [16]:
# Логистическая функция потерь
def calc_logloss(y, y_pred):
    cond_pred = np.where(y_pred == 0 , 1e-16, y_pred) # заменяем нули   
    err = np.mean(- y * np.log(cond_pred) - (1.0 - y) * np.log(1.0 - cond_pred))    
    return err

In [17]:
# Пример применения
y_true = np.array([0.0, 0.1])
y_pred = np.array([0.1, 0.0])
calc_logloss(y_true, y_pred)

1.89474833222415

2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма. Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss не превышал значение 0.3 (0.1). Как изменились веса?

In [18]:
# логистическая регрессия
def eval_LR_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot((y_pred - y), X))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

In [21]:
def standard_scaler(x):
    res = (x - x.mean()) / x.std()
    return res

In [28]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика

In [29]:
X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])
X_st[:, 2]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

In [60]:
w = eval_LR_model(X_st, y, 200000, alpha=1e-1)

20000 [-16.2427091   -1.86940882  -3.90475531  13.50901763] 0.20874421725365058
40000 [-23.32991383  -2.48704062  -6.00960348  19.2582858 ] 0.1641585574391258
60000 [-28.46500332  -2.94026656  -7.50582736  23.41651039] 0.14096383462694595
80000 [-32.5184265   -3.29804548  -8.67615891  26.68956031] 0.12659210791842251
100000 [-35.8933164   -3.59426199  -9.64639296  29.4060647 ] 0.1166706079110005
120000 [-38.80669018  -3.84767407 -10.48265646  31.74315795] 0.109302883329543
140000 [-41.38751511  -4.06963396 -11.22360001  33.80618851] 0.103539318715441
160000 [-43.71819424  -4.2674961  -11.89360808  35.66252022] 0.0988527032489802
180000 [-45.85436158  -4.44628574 -12.50897306  37.35767751] 0.09492676884669478
200000 [-47.83523093  -4.60958703 -13.08107424  38.92379106] 0.09156004715343083


При кол-ве итераций равном 200000 ошибка становится меньше 0.1. То есть увеличение кол-ва итераций уменьшает ошибку.
Уменьшение alpha приводит к увеличению ошибки. Веса в абсолютном выражении увеличились.

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred_proba).

In [61]:
def calc_pred_proba(X, w):
    y_pred = sigmoid(np.dot(X_st, w))
    return y_pred

In [65]:
y_pred_proba = calc_pred_proba(X_st, w)
y_pred_proba

array([0.33027502, 0.00223262, 1.        , 0.03215065, 0.96673777,
       0.01491352, 1.        , 0.00807259, 0.65518501, 1.        ])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, а также порог вероятности, на выходе - массив y_pred).

In [68]:
def calc_pred(X, w, trim):
    y_pred = calc_pred_proba(X, w)
    return np.where(y_pred > trim, 1, 0)    

In [69]:
y_pred = calc_pred(X_st, w, 0.5)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

5. (*) Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.

In [71]:
accuracy = 100.0 - np.mean(np.abs(y_pred - y)*100.0)
print("Точность", accuracy)

Точность 100.0


In [72]:
def confusion_matrix(y, y_pred):
    tp, fp, fn, tn = 0, 0, 0, 0
    for i, j in zip(y_pred, y):
        if i == 1:
            if j == 1:
                tp += 1
            else:
                fp += 1
        else:
            if j == 1:
                fn += 1
            else:
                tn += 1
    return np.array([[tp, fp], [fn, tn]])

In [73]:
my_confusion_matrix = confusion_matrix(y, y_pred)
my_confusion_matrix

array([[5, 0],
       [0, 5]])

In [74]:
def precision(y, y_pred):
    matrix = confusion_matrix(y, y_pred)
    tp, fp = matrix[0]
    return tp / (tp + fp)

In [75]:
my_precision = precision(y, y_pred)
my_precision

1.0

In [77]:
def recall(y, y_pred):
    matrix = confusion_matrix(y, y_pred)
    tp, fp = matrix[0]
    fn, tn = matrix[1]
    return tp / (tp + fn)

In [78]:
my_recall = recall(y, y_pred)
my_recall

1.0

In [79]:
def f1(y, y_pred):
    prec = precision(y, y_pred)
    rec = recall(y, y_pred)
    return 2 * prec * rec / (prec + rec)

In [81]:
my_f1 = f1(y, y_pred)
my_f1

1.0

6. (*) Могла ли модель переобучиться? Почему?

Да, могла. Переобучение могло произойти из-за большого кол-ва итераций. Модель подстроилась. Нужно обучающую выборку разбивать на части.